In [2]:
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from PIL import ImageTk,Image
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

win=Tk()
win.state("zoomed")
win.resizable(width=False,height=False)
'''canvas=Canvas(win)
img=ImageTk.PhotoImage(Image.open('c:/users/drishya/Desktop/download.jpeg').resize((180,130)))
canvas.create_image(0,0,anchor=NW,image=img)
canvas.pack(expand=YES,fill=BOTH) '''      


win.configure(bg="black")
win.title("MASKINDIA")

lbl_title=Label(win,text="MASKINDIA",font=('',50,'underline'),bg="black",fg="white")
lbl_title.place(relx=.40,rely=0.02)


def home_screen():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    lbl_user=Label(frm,text="-By Team MaskIndia",font=('',20,'bold'),bg='black',fg="white")
    lbl_user.place(relx=.8,rely=.1)
    
    lbl_user=Label(frm,text="Username",font=('',20,'bold'),bg='black',fg="white")
    lbl_user.place(relx=.3,rely=.2)

    entry_user=Entry(frm,font=('',20,'bold'),bd=10)
    entry_user.place(relx=.42,rely=.2)
    entry_user.focus()

    lbl_pass=Label(frm,text="Password",font=('',20,'bold'),bg='black',fg="white")
    lbl_pass.place(relx=.3,rely=.3)

    entry_pass=Entry(frm,font=('',20,'bold'),bd=10,show="*")
    entry_pass.place(relx=.42,rely=.3)

    btn_login=Button(frm,command=lambda:welcome_screen(entry_user,entry_pass),text="login",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.46,rely=.4)
    
    
def welcome_screen(entry_user=None,entry_pass=None):
    if(entry_user!=None and entry_pass!=None):
        user=entry_user.get()
        pwd=entry_pass.get()
    else:
        user="admin"
        pwd="admin"
    if(len(user)==0 or len(pwd)==0):
        messagebox.showwarning("validation","Please fill both fields")
        return
    else:
        if(user=="admin" or pwd=="admin"):
            frm=Frame(win,bg='black')
            frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
            

            btn_bulk=Button(frm,command=lambda:welcome(),text="Mask Detection",font=('',20,'bold'),bd=10,width=25)
            btn_bulk.place(relx=.35,rely=.2)
            
            btn_single=Button(frm,command=lambda:welcome_screen_new(),text="Face Detection",font=('',20,'bold'),bd=10,width=25)
            btn_single.place(relx=.35,rely=.3)
            
            #btn_bulk=Button(frm,command=lambda:welcome_screen_new(),text="Face Detection",font=('',20,'bold'),bd=10,width=25)
            #btn_bulk.place(relx=.35,rely=.4)

            btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
            btn_logout.place(relx=.9,rely=0)
        else:
            messagebox.showerror("Fail","Invalid Username/Password")

'''--------------------------------------MASK DETECTION----------------------------------------------------------------------'''            

def welcome(entry_user=None,entry_pass=None):
    
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    

    lbl_title=Label(frm,text="Mask Detection",font=('',50,'underline'),bg='black',fg="white")
    lbl_title.place(relx=.34,rely=0.04)

            
    btn_bulk=Button(frm,command=lambda:Detect_Mask(),text="Using Webcam",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.3)
    
    btn_bulk=Button(frm,command=lambda:Study_Mask(),text="MaskIndia Study",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.4)

    btn_logout=Button(frm,command=lambda:welcome_screen(),text="Back",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.1,rely=0)


    btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.9,rely=0)
    
    
    
def Detect_Mask():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    cascPath = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)
    model = load_model("mask_recog_ver2.h5") 
    video_capture = cv2.VideoCapture(0)
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray,
                                             scaleFactor=1.1,
                                             minNeighbors=5,
                                             minSize=(60, 60),
                                             flags=cv2.CASCADE_SCALE_IMAGE)
        faces_list=[]
        preds=[]
        for (x, y, w, h) in faces:
            face_frame = frame[y:y+h,x:x+w]
            face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
            face_frame = cv2.resize(face_frame, (224, 224))
            face_frame = img_to_array(face_frame)
            face_frame = np.expand_dims(face_frame, axis=0)
            face_frame =  preprocess_input(face_frame)
            faces_list.append(face_frame)
            if len(faces_list)>0:
                preds = model.predict(faces_list)
            for pred in preds:
                (mask, withoutMask) = pred
            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
            cv2.putText(frame, label, (x, y- 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2) 
            cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
            # Display the resulting frame
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()
    cv2.destroyAllWindows()


    def stop():
        cap.release()
        lmain.destroy()
        imageFrame.destroy()

    def startfac():
        global flag
        flag=True

    def stopface():
        global flag
        flag=False

    def show_frame():
        _, frame = cap.read()
        cv2image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
        faces=cascPath.detectMultiScale(gray,1.3,7)
        for x,y,w,h in faces:
            if(flag==True):
                cv2.rectangle(cv2image,(x,y),(x+w,y+h),(255,0,0),2)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, show_frame) #calls show_frame after 10 mills

    def start():
        global lmain,cap,lmain,imageFrame
        cap=cv2.VideoCapture(0)
        imageFrame=Frame(width=1500,height=2000,bd=1,bg='green')
        lmain=Label(imageFrame)
        lmain.grid(row=5, column=10)
        imageFrame.place(relx=.3,rely=.3)
        btn_DetectFace=Button(imageFrame,command=lambda:startfac(),text='detect face',font=('',20,'bold'),bd=10,width=12).place(relx=.1,rely=.9)
        btn_Stop=Button(imageFrame,command=lambda:stopface(),text='stop detecting',font=('',20,'bold'),bd=10,width=12).place(relx=.5,rely=.9)

        show_frame()
        

''''''''''''''''''''
 

    
def logout():
    option=messagebox.askyesno('Confirmation','Are you sure you want to logout?')
    if(option==True):
        home_screen()
    else:
        pass

            
        
def Study_Mask():
    
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    

    lbl_title=Label(frm,text="MaskIndia Study",font=('',50,'underline'),bg='black',fg="white")
    lbl_title.place(relx=.34,rely=0.04)
            
    btn_single=Button(frm,command=lambda:age_group(),text="Age Group Affected",font=('',20,'bold'),bd=10,width=25)
    btn_single.place(relx=.35,rely=.2)

    btn_bulk=Button(frm,command=lambda:people_wearing_mask(),text="How many people wear a Mask?",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.3)
            
    btn_bulk=Button(frm,command=lambda:male_or_female(),text="Male or Female?",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.4)
    
    btn_bulk=Button(frm,command=lambda:type_of_mask(),text="Type of Mask",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.5)
    
    btn_bulk=Button(frm,command=lambda:skin(),text="Skin Problems",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.6)

    btn_logout=Button(frm,command=lambda:welcome(),text="Back",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.1,rely=0)

    btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.9,rely=0)
    
'''--------------------------------------------age group-------------------------------------------------------------------'''   
def age_group():
    win.title('Video Player')
    frm=Frame(win,bg='white')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)   
    def snd1():
        os.system("C:/Users/Drishya/OneDrive/Desktop/age_video.mp4")
    var = IntVar()
    rb1 = Radiobutton(frm, text= "Play Video", variable = var, value=1, command=snd1)
    rb1.pack(anchor = W)

    win.title('Female')
    def snd2():
        os.system("C:/Users/Drishya/OneDrive/Desktop/age_group.png")
    var = IntVar()
    rb1 = Radiobutton(frm, text= "Female", variable = var, value=1, command=snd2)
    rb1.pack(anchor = W)

    win.title('Male')
    def snd1():
        os.system("C:/Users/Drishya/OneDrive/Desktop/major_project/download.mp4")
    var = IntVar()
    rb1 = Radiobutton(frm, text= "Male", variable = var, value=1, command=snd1)
    rb1.pack(anchor = W)


    text2 = Text(frm, height=50, width=250)
    scroll = Scrollbar(frm, command=text2.yview)
    text2.configure(yscrollcommand=scroll.set)
    text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
    text2.tag_configure('big', font=('Verdana', 20, 'bold'))
    text2.tag_configure('color',
                        foreground='#476042',
                        font=('Tempus Sans ITC', 12, 'bold'))
    text2.tag_bind('follow',
                   '<1>',
                   lambda e, t=text2: t.insert(END, "Not now, maybe later!"))
    text2.insert(END,'\n A Detailed Analysis\n', 'big')
    quote = """
    Female or Male?\n
    After the collection and analysis of data it was found that people of age group 60-74 have died the most and least are 0-14 showing 
    covid affects elders  mostly.
    """
    text2.insert(END, quote, 'color')
    text2.insert(END, 'follow-up\n', 'follow')
    text2.pack(side=LEFT)
    scroll.pack(side=RIGHT, fill=Y)

    scroll_bar = Scrollbar(frm)

    # Pack the scroll bar
    # Place it to the right side, using tk.RIGHT
    scroll_bar.pack(side=RIGHT)

    image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/total death.png")
    image = image1.resize((450, 350), Image.ANTIALIAS)
    test = ImageTk.PhotoImage(image)

    label1 = Label(image=test)
    label1.image = test

    # Position image
    label1.place(x=20, y=450)
    
    
    btn_logout=Button(frm,command=lambda:Study_Mask(),text="Back",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.66,rely=.7)
    
    
    
    
    
    
'''----------------------------------------how many people wear a mask?------------------------------------------------------'''
def people_wearing_mask():
    import tkinter as tk
    import os
    from PIL import Image, ImageTk
    frm=Frame(win,bg='white')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)   

    text2 = Text(frm, height=50, width=250)
    scroll = Scrollbar(frm, command=text2.yview)
    text2.configure(yscrollcommand=scroll.set)
    text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
    text2.tag_configure('big', font=('Verdana', 20, 'bold'))
    text2.tag_configure('color',
                        foreground='#476042',
                        font=('Tempus Sans ITC', 12, 'bold'))
    text2.tag_bind('follow',
                   '<1>',
                   lambda e, t=text2: t.insert(END, "Not now, maybe later!"))
    text2.insert(END,'\n A Detailed Analysis\n', 'big')
    quote = """
    How many people wear a mask?\n
    After the collection and analysis of data for 10 countries, on how many people are wearing masks during these Covid-19 times, we found that in Vietnam people are wearing masks whenever they go out
    and are aware of the fact that mask protects us, whereas UK stands last in this.
    Talking about INDIA, around 76% people wear a mask. They have some misconceptions as well that have been plotted with the help of graphs.

    """
    text2.insert(END, quote, 'color')
    text2.insert(END, 'follow-up\n', 'follow')
    text2.pack(side=LEFT)
    scroll.pack(side=RIGHT, fill=Y)

    scroll_bar = Scrollbar(frm)

    # Pack the scroll bar
    # Place it to the right side, using tk.RIGHT
    scroll_bar.pack(side=RIGHT)

    image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/cat5.png")
    image = image1.resize((850, 420), Image.ANTIALIAS)
    test = ImageTk.PhotoImage(image)

    label1 = Label(image=test)
    label1.image = test

    # Position image
    label1.place(x=0, y=350)

    btn_login=Button(win,command=lambda:first(),text="Other Graphs",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.66,rely=.7)
    btn_login=Button(win,command=lambda:Study_Mask(),text="Home",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.66,rely=.8)


    def first():
        frm=Frame(win,bg='white')
        frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
        text2 = Text(frm, height=50, width=250)
        scroll = Scrollbar(frm, command=text2.yview)
        text2.configure(yscrollcommand=scroll.set)
        text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
        text2.tag_configure('big', font=('Verdana', 20, 'bold'))
        text2.tag_configure('color',foreground='#476042',font=('Tempus Sans ITC', 12, 'bold'))
        quote = """
        According to the first question,"I don't see the value of wearing of a mask if I am not sick!", we found that Australians think this the most around
        46% of them.\n
        In India about 15% people think this.
        """
        text2.insert(END, quote, 'color')
        text2.insert(END, 'follow-up\n', 'follow')
        text2.pack(side=LEFT)

        image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/cat1.png")
        image = image1.resize((850, 420), Image.ANTIALIAS)
        test = ImageTk.PhotoImage(image)
        label1 = Label(image=test)
        label1.image = test
        # Position image
        label1.place(x=0, y=350)

        btn_login=Button(frm,command=lambda:second(),text="Second",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.5)
        
        btn_login=Button(win,command=lambda:people_wearing_mask(),text="Back",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.72)
        

    def second():
        frm=Frame(win,bg='white')
        frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
        text2 = Text(frm, height=50, width=250)
        scroll = Scrollbar(frm, command=text2.yview)
        text2.configure(yscrollcommand=scroll.set)
        text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
        text2.tag_configure('big', font=('Verdana', 20, 'bold'))
        text2.tag_configure('color',foreground='#476042',font=('Tempus Sans ITC', 12, 'bold'))
        quote = """
        According to the second question,"Even if I wanted to wear a mask I have heard they are not available!", we found that Japanese people think this the most around
        57% of them.\n
        In India about 35% people think this.
        """
        text2.insert(END, quote, 'color')
        text2.insert(END, 'follow-up\n', 'follow')
        text2.pack(side=LEFT)

        image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/cat2.png")
        image = image1.resize((850, 420), Image.ANTIALIAS)
        test = ImageTk.PhotoImage(image)
        label1 = Label(image=test)
        label1.image = test
        # Position image
        label1.place(x=0, y=350)

        btn_login=Button(frm,command=lambda:third(),text="Third",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.6)

        btn_login=Button(frm,command=lambda:first(),text="Back",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.7)

    def third():
        frm=Frame(win,bg='white')
        frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
        text2 = Text(frm, height=50, width=250)
        scroll = Scrollbar(frm, command=text2.yview)
        text2.configure(yscrollcommand=scroll.set)
        text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
        text2.tag_configure('big', font=('Verdana', 20, 'bold'))
        text2.tag_configure('color',foreground='#476042',font=('Tempus Sans ITC', 12, 'bold'))
        quote = """
        According to the third question,"People should wear a face mask in public even if the pandemic is over!", we found that Vietnamese people think this the most around
        87% of them.\n
        In India about 72% people think this.
        """
        text2.insert(END, quote, 'color')
        text2.insert(END, 'follow-up\n', 'follow')
        text2.pack(side=LEFT)

        image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/cat3.png")
        image = image1.resize((850, 420), Image.ANTIALIAS)
        test = ImageTk.PhotoImage(image)
        label1 = Label(image=test)
        label1.image = test
        # Position image
        label1.place(x=0, y=350)

        btn_login=Button(frm,command=lambda:fourth(),text="Fourth",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.6)

        btn_login=Button(frm,command=lambda:second(),text="Back",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.7)


    def fourth():
        frm=Frame(win,bg='white')
        frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
        text2 = Text(frm, height=50, width=250)
        scroll = Scrollbar(frm, command=text2.yview)
        text2.configure(yscrollcommand=scroll.set)
        text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
        text2.tag_configure('big', font=('Verdana', 20, 'bold'))
        text2.tag_configure('color',foreground='#476042',font=('Tempus Sans ITC', 12, 'bold'))
        quote = """
        According to the first question,"I expect people around me to wear a mask so I do not have to get sick!", we found that Japanese think this the most around
        58% of them.\n
        In India about 50% people think this.
        """
        text2.insert(END, quote, 'color')
        text2.insert(END, 'follow-up\n', 'follow')
        text2.pack(side=LEFT)

        image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/cat4.png")
        image = image1.resize((850, 420), Image.ANTIALIAS)
        test = ImageTk.PhotoImage(image)
        label1 = Label(image=test)
        label1.image = test
        # Position image
        label1.place(x=0, y=350)
        btn_login=Button(frm,command=lambda:fifth(),text="Fifth",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.6)

        btn_login=Button(frm,command=lambda:third(),text="Back",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.7)


    def fifth():
        frm=Frame(win,bg='white')
        frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
        text2 = Text(frm, height=50, width=250)
        scroll = Scrollbar(frm, command=text2.yview)
        text2.configure(yscrollcommand=scroll.set)
        text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
        text2.tag_configure('big', font=('Verdana', 20, 'bold'))
        text2.tag_configure('color',foreground='#476042',font=('Tempus Sans ITC', 12, 'bold'))
        quote = """
        According to the first question,"Total number of people wearing a mask!", we found that Vietnamese people wear mask the most around
        91% of them.\n
        In India about 76% people think this.
        """
        text2.insert(END, quote, 'color')
        text2.insert(END, 'follow-up\n', 'follow')
        text2.pack(side=LEFT)

        image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/cat5.png")
        image = image1.resize((850, 420), Image.ANTIALIAS)
        test = ImageTk.PhotoImage(image)
        label1 = Label(image=test)
        label1.image = test
        # Position image
        label1.place(x=0, y=350)    

        btn_login=Button(frm,command=lambda:fourth(),text="Back",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.6)
        
        btn_login=Button(win,command=lambda:Study_Mask(),text="Home",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.8)

    
    
'''----------------------------------------end of how many people wear a mask?------------------------------------------------------'''
    
'''----------------------------------------Male-Female Death percenatge--------------------------------------------------------------'''
def male_or_female():
    frm=Frame(win,bg='white')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
    text2 = Text(frm, height=50, width=250)
    scroll = Scrollbar(frm, command=text2.yview)
    text2.configure(yscrollcommand=scroll.set)
    text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
    text2.tag_configure('big', font=('Verdana', 20, 'bold'))
    text2.tag_configure('color',
                        foreground='#476042',
                        font=('Tempus Sans ITC', 12, 'bold'))
    text2.tag_bind('follow',
                   '<1>',
                   lambda e, t=text2: t.insert(END, "Not now, maybe later!"))
    text2.insert(END,'\n A Detailed Analysis\n', 'big')
    quote = """
        Death percentage of males in different countries?\n
        After the collection and analysis of data on what is the death percentage of males in different countries during these Covid-19 times, it was found
        that most of them around 74.01 percent died in Afganistan and might not have wore a mask and followed the rules.Whereas in Vietnam the death percenatge 
        of males were the least about 37.17%. In India 64% males died due to Covid!
        """
    text2.insert(END, quote, 'color')
    text2.insert(END, 'follow-up\n', 'follow')
    text2.pack(side=LEFT)

    image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/male_mask.png")
    image = image1.resize((950, 450), Image.ANTIALIAS)
    test = ImageTk.PhotoImage(image)

    label1 = Label(image=test)
    label1.image = test

    # Position image
    label1.place(x=0, y=350)

    btn_login=Button(frm,command=lambda:female(),text="Female",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.66,rely=.5)
    btn_login=Button(frm,command=lambda:Study_Mask(),text="Home",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.66,rely=.7)


    
    def female():
        frm=Frame(win,bg='white')
        frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
        text2 = Text(frm, height=50, width=250)
        scroll = Scrollbar(frm, command=text2.yview)
        text2.configure(yscrollcommand=scroll.set)
        text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
        text2.tag_configure('big', font=('Verdana', 20, 'bold'))
        text2.tag_configure('color',
                            foreground='#476042',
                            font=('Tempus Sans ITC', 12, 'bold'))
        quote = """
        Death percentage of Females in different countries?\n
        After the collection and analysis of data on what is the death percentage of females in different countries during these Covid-19 times, it was found
        that most of them around 62.86 percent died in Vietnam and might not have wore a mask and followed the rules.Whereas in Bangladesh the death percenatge 
        of males were the least about 23%.

        In India 36% males died due to Covid!
        """
        text2.insert(END, quote, 'color')
        text2.insert(END, 'follow-up\n', 'follow')
        text2.pack(side=LEFT)

        image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/female_mask.png")
        image = image1.resize((950, 450), Image.ANTIALIAS)
        test = ImageTk.PhotoImage(image)

        label1 = Label(image=test)
        label1.image = test

        btn_login=Button(win,text="Back",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.7)

        # Position image
        label1.place(x=0, y=350)
        
        
        btn_login=Button(win,command=lambda:summary(),text="Summary",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.5)
        btn_login=Button(win,command=lambda:male_or_female(),text="Back",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.7)
        
        
        
    def summary():
        frm=Frame(win,bg='white')
        frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
        text2 = Text(frm, height=50, width=250)
        scroll = Scrollbar(frm, command=text2.yview)
        text2.configure(yscrollcommand=scroll.set)
        text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
        text2.tag_configure('big', font=('Verdana', 20, 'bold'))
        text2.tag_configure('color',
                            foreground='#476042',
                            font=('Tempus Sans ITC', 12, 'bold'))
        quote = """
        Death percentage of Male and Female in India\n
        After the collection and analysis of data on what is the death percentage of males and females in India during these Covid-19 times, it was found
        that 57.54% of them were male and 40.086 were female so this shows that males have died more in compariosn to female.
                """
        text2.insert(END, quote, 'color')
        text2.insert(END, 'follow-up\n', 'follow')
        text2.pack(side=LEFT)

        image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/male_female_mask.png")
        image = image1.resize((550, 450), Image.ANTIALIAS)
        test = ImageTk.PhotoImage(image)

        label1 = Label(image=test)
        label1.image = test

        btn_login=Button(win,command=lambda:Study_Mask(),text="Home",font=('',20,'bold'),bd=10,width=10)
        btn_login.place(relx=.66,rely=.7)

        # Position image
        label1.place(x=0, y=350)
        

    

'''----------------------------------------male-female-done--------------------------------------------------------------------'''
    
'''----------------------------------------skin problems-----------------------------------------------------------------------'''
def skin():
    import tkinter as tk
    import os
    from PIL import Image, ImageTk
    frm=Frame(win,bg='white')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    text2 = Text(frm, height=50, width=250)
    scroll = Scrollbar(frm, command=text2.yview)
    text2.configure(yscrollcommand=scroll.set)
    text2.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
    text2.tag_configure('big', font=('Verdana', 20, 'bold'))
    text2.tag_configure('color',
                        foreground='#476042',
                        font=('Tempus Sans ITC', 12, 'bold'))
    text2.tag_bind('follow',
                   '<1>',
                   lambda e, t=text2: t.insert(END, "Not now, maybe later!"))
    text2.insert(END,'\n A Detailed Analysis\n', 'big')
    quote = """
    Types of problems by wearing a mask?\n
    After the collection and analysis of data ( around 226 people participated in the survey) for India, we found that there are some skin problems as well as other problems caused
    by wearing masks.
    Namely pimples/acne, irritations, rashes, suffocation, breathing etc. Amongst them suffocation is the most common problem and rashes and irritations are some common skin diseases.
    Around 69% of people do not face any problem while wearing a mask.
    """
    text2.insert(END, quote, 'color')
    text2.insert(END, 'follow-up\n', 'follow')
    text2.pack(side=LEFT)
    scroll.pack(side=RIGHT, fill=Y)

    scroll_bar = Scrollbar(win)

    # Pack the scroll bar
    # Place it to the right side, using tk.RIGHT
    scroll_bar.pack(side=RIGHT)

    image1 = Image.open("C:/Users/Drishya/OneDrive/Desktop/major_project/skin problems.png")
    image = image1.resize((850, 420), Image.ANTIALIAS)
    test = ImageTk.PhotoImage(image)

    label1 = Label(image=test)
    label1.image = test

    # Position image
    label1.place(x=0, y=350)

    btn_login=Button(win,command=lambda:Study_Mask(),text="Back",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.66,rely=.7)
    
'''----------------------------------------skin problems end------------------------------------------------------------------'''   
'''-----------------------------------code for face detection starts here----------------------------------------------------'''


def welcome_screen_new(entry_user=None,entry_pass=None):
    
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    

    lbl_title=Label(frm,text="Face Detection",font=('',50,'underline'),bg='black',fg="white")
    lbl_title.place(relx=.34,rely=0.04)

    btn_bulk=Button(frm,command=lambda:welcome3(),text="Using Video",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.3)
            
    btn_bulk=Button(frm,command=lambda:welcome4(),text="Using Webcam",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.4)
 
    btn_logout=Button(frm,command=lambda:welcome_screen(),text="Back",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.1,rely=0)

    btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.9,rely=0)
    
            
    
def welcome2():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
    Button(frm,command=lambda:browse(),text='browse image',font=('',20,'bold'),bd=10).place(relx=.4,rely=.1)
    Label(frm,font=('',40,'bold','underline'),bg='black',fg="white").pack()
    clf=cv2.CascadeClassifier('c:/users/drishya/downloads/dataset/haar/haarcascade_frontalface_default.xml')   
    
    
    def browse():
        global imageFrame   
        imageFrame=None
        if(imageFrame!=None):
            imageFrame.destroy()
        file_path=filedialog.askopenfilename()
        frame=cv2.imread(file_path)
        cv2image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)    
        imageFrame = Frame(frm,width=700, height=500,bd=10,bg='black')
        lmain = Label(imageFrame)
        lmain.grid(row=0, column=0)
        imageFrame.place(relx=.007,rely=.2)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        Button(win,command=lambda:startface(frame,cv2image,lmain),text='detect face',font=('',20,'bold'),bd=10,width=12).place(relx=.4,rely=.9)
    
    def startface(frame,cv2image,lmain):
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
        faces=clf.detectMultiScale(gray,1.2,5)
        for x,y,w,h in faces:
            cv2.rectangle(cv2image,(x,y),(x+w,y+h),(255,0,0),2)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        
def welcome4():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
   # btn_StartCam=Button(command=lambda:start(),text='start Camera ',font=('',20,'bold'),bd=10).place(relx=.01,rely=.2)
    #btn_StopCam=Button(command=lambda:stop(),text='stop camera',font=('',20,'bold'),bd=10).place(relx=.01,rely=.4)
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)    
    flag=True
    clf=cv2.CascadeClassifier('c:/users/drishya/downloads/dataset/haar/haarcascade_frontalface_default.xml')

    def show_frame():
        _, frame = cap.read()
        cv2image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
        faces=clf.detectMultiScale(gray,1.3,7)
        for x,y,w,h in faces:
            if(flag==True):
                cv2.rectangle(cv2image,(x,y),(x+w,y+h),(255,0,0),2)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, show_frame) #calls show_frame after 10 mills



    
    
def welcome3():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
    
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
    Button(frm,command=lambda:browse(),text='browse video',font=('',20,'bold'),bd=10).place(relx=.4,rely=.1)
    Label(frm,font=('',40,'bold','underline'),bg='black',fg="white").pack()
    clf = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    #clf=cv2.CascadeClassifier('c:/users/drishya/downloads/dataset/haar/haarcascade_frontalface_default.xml')  
    
    def browse():
        global imageFrame   
        imageFrame=None
        if(imageFrame!=None):
            imageFrame.destroy()
        file_path=filedialog.askopenfilename()
        vdo=cv2.VideoCapture(file_path)
        while(True):
            flag,img=vdo.read()
            if(flag==True):
                gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                faces=clf.detectMultiScale(gray,1.4,4)
                for x,y,w,h in faces:
                    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
                cv2.imshow('vdo',img)
                k=cv2.waitKey(1)
                if(k==ord('q')):
                    break  
            else:
                break   
        vdo.release()
        cv2.destroyAllWindows()  
    

    
home_screen()    
win.mainloop() #to make window visible